In [67]:
import altair as alt
import pandas as pd
from datetime import datetime
from pony.orm import *

In [2]:
from bail.db import DB, Case
db = DB()

In [70]:
dane = db.cases_in_county(13)
everything = select(c for c in Case)
no_bond = select(c for c in Case if c.signature_bond == None and c.cash_bond == None)
bond = select(c for c in Case if c.signature_bond != None or c.cash_bond != None)
bond_no_outliers = select(c for c in Case if (c.signature_bond < 100000 or c.cash_bond < 100000) and c.filing_date > datetime(2018, 1, 1))

In [71]:
len(everything), len(no_bond), len(bond), len(bond_no_outliers)

(3987, 2168, 1819, 1730)

In [5]:
len(dane.filter(lambda c: c.race != "Caucasian"))

661

In [57]:
example_case = bond.first()
example_case.filing_date

datetime.date(2017, 7, 7)

In [72]:
def bond_type(c):
    if c.cash_bond and c.cash_bond > 0:
        return 'cash'
    else:
        return 'signature'

data = pd.DataFrame([{
    'race': c.race or 'Unknown',
    'date': c.filing_date,
    'county': c.county_number,
    'bond': c.signature_bond or c.cash_bond,
    'bond_type': bond_type(c),
    'url': c.url,
} for c in bond_no_outliers])
data["date"] = pd.to_datetime(data["date"])

In [73]:
data

,race,date,county,bond,bond_type,url
0,Caucasian,2019-04-01,4,7500.0,cash,https://wcca.wicourts.gov/caseDetail.html?case...
1,American Indian or Alaskan Native,2020-06-22,4,500.0,signature,https://wcca.wicourts.gov/caseDetail.html?case...
2,American Indian or Alaskan Native,2020-06-02,4,1500.0,cash,https://wcca.wicourts.gov/caseDetail.html?case...
3,American Indian or Alaskan Native,2020-05-12,4,500.0,cash,https://wcca.wicourts.gov/caseDetail.html?case...
4,American Indian or Alaskan Native,2018-11-26,4,2600.0,cash,https://wcca.wicourts.gov/caseDetail.html?case...
...,...,...,...,...,...,...
1725,Caucasian,2019-01-15,9,2500.0,signature,https://wcca.wicourts.gov/caseDetail.html?case...
1726,Caucasian,2020-04-14,9,1000.0,signature,https://wcca.wicourts.gov/caseDetail.html?case...
1727,Caucasian,2018-08-27,9,1000.0,cash,https://wcca.wicourts.gov/caseDetail.html?case...
1728,Caucasian,2019-05-20,9,300.0,signature,https://wcca.wicourts.gov/caseDetail.html?case...


In [74]:
alt.Chart(data).mark_point().encode(
    x='date:T',
    y='bond:Q',
    shape='bond_type:N',
    color='race:N',
    href='url'
)

alt.Chart(...)

In [80]:
brush = alt.selection_interval()

chart = alt.Chart(data).mark_point().encode(
    y='bond:Q',
    color=alt.condition(brush, 'bond_type:N', alt.value('lightgray')),
#    shape='bond_type:N',
#    color='race:N',
    href='url'
).add_selection(
    brush
)

chart.encode(x='date:T') | chart.encode(x='race:N')

alt.HConcatChart(...)